Goal here is to build a basic model, reiterate later upon study of other solutions, revisit with different valuation method

**Rough draft to be edited for clarity**

In NFL, what is the best way to distribute value across cap?

Clustering (assumptions): minimum / rookie contract / free agent / franchise tag / outliers

53+ roster spots.

How many of each contract can be afforded, and what is the optimal distribution?

Then account for positional value?


cluster by salary, Measure AV/salary for all and determine optimal distribution of salaries by contract type by AV (as a basic measure of player value)

**Import the data**

need to use webscraping. 

Use pfr and spotrac as the sources. in future I think I would use these files to make a sql database

Not sure which value to use for spotrac, average only appears available for active (2022-) contracts, historical values seem 
subject to be skewed by  things like contract restructures -- sorting by 'cash' seems to provide desired values, though this may be misleading for players whose salary is being paid by former team -- might make sense to go player by player and use yearly totals from 'career earnings', which appears to be cash values paid across all teams for a season

spotrac and pfr seem to be consistent with which players were on which rosters historically, though spotrac adds has players across categories like injury IR, dead cap, practice squad etc. and classifies based on which category a player is in at the end of the season.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
teams = ['https://www.pro-football-reference.com/teams/buf/2021_roster.htm', 'https://www.pro-football-reference.com/teams/nwe/2021_roster.htm', 'https://www.pro-football-reference.com/teams/mia/2021_roster.htm', 'https://www.pro-football-reference.com/teams/nyj/2021_roster.htm', 'https://www.pro-football-reference.com/teams/cin/2021_roster.htm', 'https://www.pro-football-reference.com/teams/pit/2021_roster.htm', 'https://www.pro-football-reference.com/teams/cle/2021_roster.htm', 'https://www.pro-football-reference.com/teams/rav/2021_roster.htm', 'https://www.pro-football-reference.com/teams/oti/2021_roster.htm', 'https://www.pro-football-reference.com/teams/clt/2021_roster.htm', 'https://www.pro-football-reference.com/teams/htx/2021_roster.htm', 'https://www.pro-football-reference.com/teams/jax/2021_roster.htm', 'https://www.pro-football-reference.com/teams/kan/2021_roster.htm', 'https://www.pro-football-reference.com/teams/rai/2021_roster.htm', 'https://www.pro-football-reference.com/teams/sdg/2021_roster.htm', 'https://www.pro-football-reference.com/teams/den/2021_roster.htm', 'https://www.pro-football-reference.com/teams/dal/2021_roster.htm', 'https://www.pro-football-reference.com/teams/phi/2021_roster.htm', 'https://www.pro-football-reference.com/teams/was/2021_roster.htm', 'https://www.pro-football-reference.com/teams/nyg/2021_roster.htm', 'https://www.pro-football-reference.com/teams/gnb/2021_roster.htm', 'https://www.pro-football-reference.com/teams/min/2021_roster.htm', 'https://www.pro-football-reference.com/teams/chi/2021_roster.htm', 'https://www.pro-football-reference.com/teams/det/2021_roster.htm', 'https://www.pro-football-reference.com/teams/tam/2021_roster.htm', 'https://www.pro-football-reference.com/teams/nor/2021_roster.htm', 'https://www.pro-football-reference.com/teams/atl/2021_roster.htm', 'https://www.pro-football-reference.com/teams/car/2021_roster.htm', 'https://www.pro-football-reference.com/teams/ram/2021_roster.htm', 'https://www.pro-football-reference.com/teams/sfo/2021_roster.htm', 'https://www.pro-football-reference.com/teams/sea/2021_roster.htm', 'https://www.pro-football-reference.com/teams/crd/2021_roster.htm', ]

In [3]:
df = pd.DataFrame()

In [4]:
for team in teams:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(team)
    soup = BeautifulSoup(driver.page_source)
    driver.quit()
    table = soup.find_all('table')[0]  #This is the index of any table of that page. If you change it you can get different tables.
    tab_data = [[celldata.text for celldata in rowdata.find_all(["th","td"])]
                            for rowdata in table.find_all("tr")]
    #for data in tab_data[1:-1]: #first and last rows are names of the columns and team totals respectively
        #print(' '.join(data))
        
    team_df = pd.DataFrame(tab_data[1:-1], columns = tab_data[0])
    #team_df = team_df.drop(30) #this row is a repeat of column names
    team_df.insert(loc=13, column='Year', value=2021)
    team_df[['No.', 'Age', 'G', 'GS', 'Wt', 'AV']] = team_df[['No.', 'Age', 'G', 'GS', 'Wt', 'AV']].apply(pd.to_numeric, errors='coerce')
    
    df = df.append(team_df)


    #create dictionary of all team names as they appear on pfr and get all tables, keep each table separate

In [5]:
df = df.reset_index()

In [6]:
df = df.drop(['index'], axis=1)

In [11]:
df = df[df['Age'].notna()]

In [13]:
df = df.reset_index()

In [15]:
df = df.drop(['index'], axis=1)

In [18]:
df['No.'] = df['No.'].fillna(0)

In [28]:
df['Wt'].iloc[1535] = 252

C:\Users\FIDataSci.DESKTOP-5QIEL80\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [30]:
df[['No.', 'Age', 'G', 'GS', 'Wt', 'AV']] = df[['No.', 'Age', 'G', 'GS', 'Wt', 'AV']].astype('int64')

In [45]:
df['No.'].replace([0], '', inplace=True)

In [46]:
df.to_csv(r'C:\Users\FIDataSci.DESKTOP-5QIEL80\profootballref-dataframe-csv', sep='\t', encoding='utf-8')

**Inspect dataset, prepare data appropriately**

Want to have dataframe that at least has player position({'str':float64}), salary(real()), value(av()), and yearly salary cap (real()) formatted properly

**Cluster salaries**

I think this is mostly done using an ai?

**Evaluate all players value compared to salary**

value_per_dollar = av / salary

**Visualize and evaluate**
analyze by clusters(look up how to cluster, pretty sure this is done with basic ai) (use mean. assumpt: In NFL, team control to block player movement prevents teams from paying above market rate, and that teams would do so to extract value. However, they can't so much of how it is distributed is (assumed)random. so each team is as likely to acquire players anywhere within that cluster of contracts.

Use salary earned rather than cap figure to account for fact that that number fluctuates to accomodate teams to their cap needs, but doesn't reflect cost as effectively as salary)

perc_cap = salary / salary_cap 

order clusters by value, how many of each can exist on a team, best distribution given need for at least 53(how many could become better understood)

**Ideas to explore**

better ways to evaluate value than av

how many players are needed on a roster, does that affect distribution?

evaluate by positional value (did it have underlying effect in previous inquiry?)

find teams historically that fit similar construction based on first inquiry, and compare their relative performance (AV, W/L, super bowl, etc)/
find all possible clusters, look for their occurences in nfl teams and compare by their relative performance (AV, W/L, super bowl, etc)

**Sources**

pfr scraping - https://stmorse.github.io/journal/pfr-scrape-python.html

spotrac scraping - https://github.com/the-data-analyst/python_portfolio_web_scraper-spotrac/blob/master/webScraping_sample.py